##Setup

In [1]:
import os
import time
from enum import IntEnum
import sys

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pyplot, colors

!pip install GenProcTrees
import GenProcTrees as gpt
import random
import PIL
from PIL import Image
from matplotlib import cm

##Tree Generation

In [2]:
def define_parameters():
  min_distances = np.arange(0.02, 0.05 ,0.005)
  max_distances = np.arange(0.2, 0.5, 0.05)
  branch_lengths = np.arange(0.01, 0.075, 0.005) # np.arange(0.045, 0.075, 0.005)
  turn_factors = np.arange(0.06, 0.64, 0.05) # np.concatenate((np.arange(0.06, 0.15, 0.03), np.arange(0.5, 0.64, 0.05)))
  leaf_starts = np.arange(0.20, 0.37, 0.03)
  number_of_leaves = np.arange(310, 400, 15, dtype=int)

  all_parameters = np.array(
    np.meshgrid(min_distances, max_distances, branch_lengths, turn_factors, leaf_starts, number_of_leaves)).T.reshape(-1, 6)
  # print(all_parameters.shape)
  return all_parameters

def define_crazy_parameters():
  MIN_VAL = 0.01
  MAX_VAL = 0.99
  # min_distances < max_distances
  # min_distances < branch_lengths < max_distances
  min_distances = np.arange(MIN_VAL, MAX_VAL / 2, 0.05)
  max_distances = np.arange(MIN_VAL, MAX_VAL / 2, 0.05)
  branch_lengths = np.arange(MIN_VAL, MAX_VAL, 0.05)
  turn_factors = np.arange(MIN_VAL, MAX_VAL, 0.05)
  leaf_starts = np.arange(MIN_VAL, MAX_VAL, 0.05)
  number_of_leaves = np.arange(310, 400, 15, dtype=int)

  all_parameters = np.array(
    np.meshgrid(min_distances, max_distances, branch_lengths, turn_factors, leaf_starts, number_of_leaves)).T.reshape(-1, 6)
  # print(all_parameters.shape)
  return all_parameters

In [3]:
def generate_trees(num_trees, all_parameters):
  tree_objects = []
  trees = []
  # print(np.random.choice(all_parameters.shape[0], size=num_trees, replace=False).shape)
  parameters = all_parameters[np.random.choice(all_parameters.shape[0], size=num_trees, replace=False)]
  for i in range(0, num_trees):
    print("Generating tree {}".format(i))
    params = parameters[i]
    np.set_printoptions(suppress=True)
    # print(np.round(params, 3))
    # print(params)
    tree = gpt.generate_tree({
      'min_distance': params[0],
      'max_distance': params[0] + params[1],
      'branch_length': params[2],
      'turn_factor': params[3],
      'leaf_start': params[4],
      'number_of_leaves': int(params[5]),
      'limit_2d': True
    })
    tree_objects.append(tree)
    # print(dir(tree))
    # print(tree.parameters)
    tree.leaves = []
    tree_name = str(i)
    if(i < 10):
      tree_name = '0' + str(i)
    path = "trees/tree" + tree_name + ".png"
    gpt.write_image_from_tree(path, tree)
    tree_array = image_to_array(path) 
    trees.append(tree_array)
    visualize_tree(tree_array, "original/original{}.png".format(tree_name))
    tree_array = place_disease(tree_array)
    visualize_tree(tree_array, "infected/infected{}.png".format(tree_name))
  trees = np.array(trees)
  return trees

In [4]:
def test_generate_tree():
  num_trees = 1
  tree_objects = []
  trees = []
  for i in range(0, num_trees):
    print("Generating tree {}".format(i))
    tree = gpt.generate_tree({'min_distance': 0.048,
                              'max_distance': 0.4,
                              'branch_length': 0.01, # 0.03
                              'turn_factor': 0.35,
                              'leaf_start': 0.30,
                              'number_of_leaves': 310,
                              'limit_2d': True})
    tree_objects.append(tree)
    # print(dir(tree))
    # print(tree.parameters)
    tree.leaves = []
    tree_name = str(i)
    if(i < 10):
      tree_name = '0' + str(i)
    path = "trees/tree" + tree_name + ".png"
    gpt.write_image_from_tree(path, tree)
    tree_array = image_to_array(path) 
    trees.append(tree_array)
    visualize_tree(tree_array, "original/original{}.png".format(tree_name))
    tree_array = place_disease(tree_array)
    visualize_tree(tree_array, "infected/infected{}.png".format(tree_name))
  trees = np.array(trees)
  tree = trees[0]
  return trees

In [5]:
def image_to_array(path):
  image = PIL.Image.open(path)
  image_array = np.asarray(image)
  image_size = image_array.shape[0]
  uniques = np.unique(image_array.reshape(-1, image_array.shape[2]), axis=0)
  tree_array = np.zeros((image_size, image_size))
  for col in range(image_size):
    for row in range(image_size):
      if(np.array_equal(image_array[col, row], uniques[1])) :
        tree_array[col][row] = 1
  crop_size = 100
  tree_array = tree_array[crop_size:image_size - crop_size, crop_size:image_size - crop_size]
  return tree_array

In [6]:
def visualize_tree(tree, path):
  tree = np.stack([tree, tree, tree], axis=2)
  uniques = np.unique(tree.reshape(-1, tree.shape[2]), axis=0)
  image_size = tree.shape[0]
  for col in range(image_size):
    for row in range(image_size):
      pixel = tree[col, row]
      if(np.array_equal(pixel, [0, 0, 0])):
        tree[col, row] = [135, 206, 235]
      elif(np.array_equal(pixel, [1, 1, 1])):
        tree[col, row] = [139, 69, 19]
      elif(np.array_equal(pixel, [2, 2, 2])):
        tree[col, row] = [255, 255, 0]
      elif(np.array_equal(pixel, [3, 3, 3])):
        tree[col, row] = [255, 0, 0]
  img = Image.fromarray(np.uint8(tree), 'RGB')
  img.save(path)

In [7]:
def place_disease(tree):
  branch_pixel_indices = np.where(tree == 1)
  # print(branch_pixel_indices[0].shape)
  disease_site_index = np.random.randint(0, branch_pixel_indices[0].shape[0])
  disease_site_x = branch_pixel_indices[0][disease_site_index]
  disease_site_y = branch_pixel_indices[1][disease_site_index]
  # print(disease_site_x, disease_site_y)
  tree[disease_site_x, disease_site_y] = 2
  return tree

In [8]:
!rm -rf trees
!rm -rf original
!rm -rf infected
!mkdir trees
!mkdir original
!mkdir infected

random.seed(time.time())

In [9]:
# all_parameters = define_crazy_parameters()
all_parameters = define_parameters()
num_trees = 20
trees = generate_trees(num_trees, all_parameters)

Generating tree 0
[  0.04    0.4     0.015   0.11    0.23  385.   ]
Generating tree 1
[  0.025   0.45    0.02    0.46    0.23  385.   ]
Generating tree 2
[  0.04   0.4    0.03   0.21   0.23 370.  ]
Generating tree 3
[  0.04    0.35    0.025   0.51    0.32  385.   ]
Generating tree 4
[  0.045   0.35    0.04    0.36    0.26  370.   ]
Generating tree 5
[  0.035   0.45    0.01    0.06    0.29  340.   ]
Generating tree 6
[  0.025   0.25    0.04    0.61    0.23  385.   ]
Generating tree 7
[  0.035   0.4     0.03    0.36    0.29  310.   ]
Generating tree 8
[  0.035   0.45    0.03    0.31    0.23  370.   ]
Generating tree 9
[  0.03   0.4    0.01   0.41   0.2  325.  ]
Generating tree 10
[  0.03    0.25    0.045   0.61    0.32  355.   ]
Generating tree 11
[  0.045   0.45    0.025   0.16    0.23  385.   ]
Generating tree 12
[  0.045   0.2     0.03    0.61    0.35  310.   ]
Generating tree 13
[  0.025   0.35    0.015   0.61    0.2   310.   ]
Generating tree 14
[  0.04    0.2     0.035   0.41    0.

In [10]:
tree = test_generate_tree()

Generating tree 0


In [11]:
import imageio
images = []
path = 'trees/'
filenames = os.listdir(path)
for filename in filenames:
    filepath = path + filename
    images.append(imageio.imread(filepath))
imageio.mimsave('/content/movie.gif', images, duration=0.5)